In [1]:
from pytube import YouTube
from pydub import AudioSegment
from pytube import Playlist
import os
import re
from glob import glob
from tqdm import tqdm
import pandas as pd
import random

/tmp/ipykernel_1840413/1090648631.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def flow():
    """
    - input the playlists for each qare2

    - logg in the qare2 and the sura (`logg_in()`)

    - check , if the qare2 (extract the name from the title using `get_reciter()`)\
      has more than 4 occurences ( using the function `get_thresh()`) for the same sura then find another sura\
      for the same qare2 using the function `random_slicer()` which slices three random links from the urls in the\
      playlist.
    
    - download the sura (keeping the name of the reciter and the sura)

    - split the sura into 100 segments (1 second each)

    - save them with the prefix containing the qare2's name and index
  

    """
pass


In [3]:
RECITERS = [
    "احمد الشلبي",
    "علاء عقل",
    "اسماعيل القاضي",
    "علاء ياسر",
    "محمد حجازي",
    "محمد نصر الدين",
    "مختار الحاج",
    "إرويانتو",
    "زين ابو كوثر",
    "Zain Abu Kautsar",
    "حسين عبد الظاهر",
    "طارق محمد",
    "عبد الباسط",
    "المنشاوي",
    "عبد العزيز سحيم",
    "المنشاوي"
]


In [4]:
df = pd.DataFrame(columns = ['reciter','sura'])

In [5]:
def get_thresh(df,reciter,sura):
    # Filter DataFrame based on the conditions
    filtered_df = df[df['reciter'] == reciter]
    if len(filtered_df[filtered_df['sura'] == sura]) > 4:
         return True

In [6]:
def get_reciter(RECITERS, string):
    matches = [substring for substring in RECITERS if substring.lower() in string.lower()]
    
    if not matches:
        # No match found
        return None  # or raise an exception, return a default value, etc.
    
    return matches[0]

def get_sura(title):
    match = re.search(r"سورة\s[الء-ي]+", title)
    
    if match is None:
        # No match found
        return None  # or raise an exception, return a default value, etc.
    
    return match.group(0)

In [7]:
def logging_in(reciter ,title,df):
    """
    logginf in the name of the reciter and the name of the sura
    """
    sura = get_sura(title)
    reciter_name = reciter
    new_row = {'reciter': reciter_name, 'sura':sura}
    print(f"logging in {reciter} for {sura}")
    df.loc[len(df)] = new_row
    return df

In [8]:
def random_slicer(input_list, num_items=3):
    """
    slices three random items from a list
    """
    # Make a copy of the input list to avoid modifying the original list
    input_list = list(input_list)
    shuffled_list = input_list.copy()

    # Shuffle the list randomly
    random.shuffle(shuffled_list)

    # Slice the first 'num_items' elements from the shuffled list
    sliced_items = shuffled_list[:num_items]

    return sliced_items

In [9]:
def get_title(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only("mp4")
    return audio_stream.title

In [10]:
def download_youtube_audio(url, output_path):
    yt = YouTube(url)
    audio_stream = yt.streams.filter(only_audio=True).all() 
    audio_stream[0].download(output_path)

In [11]:
def download_audio_from_playlist(url, output_path, df):
    try:
        playlist = Playlist(url)
        urls = playlist.video_urls

        sura = None  # Initialize sura outside the loop

        while True:
            shuffled_playlist = random_slicer(urls, 10)

            for url in shuffled_playlist:
                title = get_title(url)
                reciter = get_reciter(RECITERS, title)

                if reciter is None:
                    reciter = get_reciter(RECITERS, playlist.title)
                else:
                    sura = get_sura(title)
                    df_new = logging_in(reciter, title, df)

                    if sura is not None:
                        if get_thresh(df_new, sura, reciter):
                            print('Reslicing the playlist')
                            break  # Break out of the inner loop to re-randomize the playlist
                        else:
                            try:
                                download_youtube_audio(url, output_path)
                                print(f"Downloading {sura} for {reciter}\n Title of the video is {title}")
                            except Exception as e:
                                print(f"Error processing video: {title} - {e}")
    
    except:
        print("Couldn't return playlist")
        return title, reciter, sura, df_new


In [12]:
def split_audio(input_path, prefix, output_path, segment_length=1000, export_format="mp3", bitrate="192k"):
    try:
        audio = AudioSegment.from_file(input_path)

        # Calculate the number of segments
        num_segments = len(audio) // segment_length

        # Split the audio into segments
        for i in range(num_segments):
            start_time = i * segment_length
            end_time = (i + 1) * segment_length
            segment = audio[start_time:end_time]

            # Save each segment
            output_filename = f"{prefix}_segment_{i + 1}.{export_format}"
            segment.export(os.path.join(output_path, output_filename), format=export_format, bitrate=bitrate)
    except:
        print(f"couldn't save {output_filename}")


In [13]:
RECITERS

['احمد الشلبي',
 'علاء عقل',
 'اسماعيل القاضي',
 'علاء ياسر',
 'محمد حجازي',
 'محمد نصر الدين',
 'مختار الحاج',
 'إرويانتو',
 'زين ابو كوثر',
 'Zain Abu Kautsar',
 'حسين عبد الظاهر',
 'طارق محمد',
 'عبد الباسط',
 'المنشاوي',
 'عبد العزيز سحيم',
 'المنشاوي']

In [14]:
# List of YouTube video URLs
video_urls = {
    # 'احمد الشلبي':["https://www.youtube.com/playlist?list=PLSE-KCHIEexiq2fLNI_wXvy7GE2ZsscnA",
    #                 "https://www.youtube.com/playlist?list=PLSI-M74-EX5fYID8_WsTWa9reh7vfnogi",
    #                 "https://www.youtube.com/playlist?list=PLymw_19SrMidvYi90juk3V33OqyJHYByj",
    #                 "https://www.youtube.com/playlist?list=PLKjpxvaXlINGQvbnjZn1lMkRWBq0GMhA-"],

    # 'علاء عقل':["https://www.youtube.com/playlist?list=PLkFm8Qv5Xhk7ayASXyeWJpnwsnP6sQMaO",
    #             "https://www.youtube.com/playlist?list=PLnQ6oMsNoobhr1UuGzQAK7KOxmSWGLu0l",
    #             "https://www.youtube.com/playlist?list=PLSI-M74-EX5ceyQHpDcpciTBuOcMq2DRH",
    #             "https://www.youtube.com/playlist?list=PLs6BqX7AeetBLzlOWLIrYuk5MzGSaHBmQ"],

    # 'اسماعيل القاضي':["https://www.youtube.com/playlist?list=PLT2DqdbNqk6sprS-AxI-AwhoqH2aY37hH",
    #                   "https://www.youtube.com/playlist?list=PLT2DqdbNqk6tXDOaTkW6i1IemJ_6WhDJJ",
    #                   "https://www.youtube.com/playlist?list=PLT2DqdbNqk6tHX8uZRAPll5CSMCWU7YGa"], #-> add more 
                      
    # 'علاء ياسر':["https://www.youtube.com/playlist?list=PLymw_19SrMic6PCzakS6Sj31HxdL6z-Fr",
    #              "https://www.youtube.com/playlist?list=PLnQ6oMsNoobiDt_YwDOauNphVyS86H5tC",
    #              "https://www.youtube.com/playlist?list=PLSI-M74-EX5fpJcJJArGXhBtARr-w05a5",
    #              "https://www.youtube.com/playlist?list=PLKjpxvaXlINF-1WaHRvbVm2665qKrAwIX"],

    # 'مختار الحاج':["https://www.youtube.com/playlist?list=PLnQ6oMsNoobhVkeaQsHr5T2l0tqxub5-C",
    #                "https://www.youtube.com/playlist?list=PLymw_19SrMifJH4O8B7vUl01El0dFWcDx",
    #                "https://www.youtube.com/playlist?list=PLUScscfmuunlW7U2K_ceSJXhhiAc-bL_R"],
                   
    'زين ابو كوثر':["https://www.youtube.com/playlist?list=PLV0CS4cp77rk2J32N9bOD5IRgYGntUrL6",
                    "https://www.youtube.com/playlist?list=PLWUE7RQCHIjI9Jb9SpxsnMAUAFXJzCf7a",
                    "https://www.youtube.com/playlist?list=PLM7w6q4q6WlPUVU_epVL5w-CWKXLM5CKt"]

}

for name , playlists in video_urls.items():
    os.makedirs(f"data/{name}", exist_ok=True)
    print(f'creating directory for {name}')
    for playlist in playlists:
             
        relative_output_dir_path = os.path.relpath(f"data/{name}")

        title,reciter,sura,_ = download_audio_from_playlist(playlist, relative_output_dir_path, df)

        videos_for_conv_paths = glob(os.path.join(relative_output_dir_path,"*.mp4"))
        for i,video in tqdm(enumerate(videos_for_conv_paths)):

            prefix = f"{title[:-1]}_{reciter}_{sura}_{i + 1}"

            split_audio(video,prefix, relative_output_dir_path)
            
        for file in videos_for_conv_paths:
            # if filename.endswith(".mp4"):
            #     file_path = os.path.join(relative_output_dir_path, filename)
                os.remove(file)
                print(f"Deleted: {file}")
        print("Restarting")
        

creating directory for زين ابو كوثر


logging in Zain Abu Kautsar for None
logging in Zain Abu Kautsar for سورة الليل


/tmp/ipykernel_1840413/1969732401.py:3: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  audio_stream = yt.streams.filter(only_audio=True).all()


 Title of the video is Surah Al-Lail سورة الليل - Zain Abu Kautsar | New Recitation
logging in Zain Abu Kautsar for سورة النازعات


/tmp/ipykernel_1840413/1969732401.py:3: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  audio_stream = yt.streams.filter(only_audio=True).all()


...

0it [00:00, ?it/s]

Restarting


logging in زين ابو كوثر for None
logging in زين ابو كوثر for None
logging in زين ابو كوثر for None
logging in زين ابو كوثر for None
logging in زين ابو كوثر for None
logging in زين ابو كوثر for None
Couldn't return playlist


0it [00:00, ?it/s]

Restarting


In [15]:
df

,reciter,sura
0,Zain Abu Kautsar,None
1,Zain Abu Kautsar,سورة الليل
2,Zain Abu Kautsar,سورة النازعات
3,Zain Abu Kautsar,سورة الطارق
4,Zain Abu Kautsar,سورة البلد
...,...,...
58,زين ابو كوثر,None
59,زين ابو كوثر,None
60,زين ابو كوثر,None
61,زين ابو كوثر,None
